In [4]:
import numpy as np
import scipy.linalg

class Simulation:
    def __init__(self, arrival_rate, service_rate):
        self.num_in_system = 0
        self.clock         = 0.0
        self.arrival_rate  = arrival_rate
        self.service_rate  = service_rate
        self.t_arrival     = self.generate_interarrival()
        self.t_depart      = float('inf')
        self.num_arrivals  = 0
        self.num_departs   = 0
        self.total_wait    = 0.0
        

    def advance_time(self):
        t_event             = min(self.t_arrival, self.t_depart)
        self.total_wait     += self.num_in_system * (t_event - self.clock)
        self.clock          = t_event
        if self.t_arrival <= self.t_depart :
            self.handle_arrival_event()
        else:
            self.handle_depart_event()

    def handle_arrival_event(self):
        self.num_in_system      += 1
        self.num_arrivals       += 1
        if self.num_in_system <= 1:
            self.t_depart   = self.clock  + self.generate_servicetime()
        self.t_arrival          = self.clock    + self.generate_interarrival()
        
    def handle_depart_event(self):
        self.num_in_system      -= 1
        self.num_departs        += 1
        if self.num_in_system > 0 :
            self.t_depart       = self.clock    + self.generate_servicetime()
        else:
            self.t_depart       = float('inf')

    def generate_interarrival(self):
        return np.random.exponential(1./self.arrival_rate)

    def generate_servicetime(self):
        return np.random.exponential(1./self.service_rate)



In [8]:
class MarkovChain:
    def __init__(self):
        
        self.state = {
                        0 : "Burger",
                        1 : "Pizza",
                        2 : "Hotdog"
                     }
        self.A = np.array( [
                [0.2, 0.6, 0.2], 
                [0.3, 0.0, 0.7], 
                [0.5, 0.0, 0.5]
                        ] )
    def Random_Walf(self):
        n = 15
        start_state = 0
        curr_state = start_state
        print(state[curr_state], "--->", end=" ")
        while n-1:
            curr_state = np.random.choice([0, 1, 2], p=A[curr_state])
            print(state[curr_state], "--->", end=" ")
            n-=1
        print("stop")
    def Monte_Carlo(self):
        # Approach -1 : Monte Carlo 
        steps = 10**6
        start_state = 0
        curr_state = start_state
        pi = np.array([0, 0, 0])
        pi[start_state] = 1
        i = 0
        while i<steps:
            curr_state = np.random.choice([0,1,2], p=A[curr_state])
            pi[curr_state]+=1
            i +=1
        print("π = ", pi/steps)
        return pi/steps
    def Matrix_Multiplication(self):
        # Approach - 2 :  Repeated Matrix Multiplication
        steps = 10**3
        A_n = A
        i=0
        while i<steps:
            A_n =  np.matmul(A_n, A)
            i+=1

        print("A^n = \n", A_n, "\n")
        print("π = ", A_n[0])
        return A_n[0]
    def Eigen_Vectors(self):
        # Approach - 3 : Finding Left Eigen Vectors
        values, left = scipy.linalg.eig(A, right = False, left = True)
        # print("left eigen vectors = \n", left, "\n")
        # print("eigen values = \n", values)
        pi = left[:,0]
        pi_normalized = [(x/np.sum(pi)).real for x in pi]
        return pi_normalized

    def find_prob(self,seq, A, pi):
        start_state = seq[0]
        prob = pi[start_state]
        prev_state, curr_state = start_state, start_state
        for i in range(1, len(seq)):
            curr_state = seq[i]
            prob *= A[prev_state][curr_state]
            prev_state = curr_state
        return prob
    
class QMG1:
    def __init__(self,arrival_rate, service_rate):
        self.arrival_rate   = arrival_rate
        self.service_rate   = service_rate
    def getProb(self,n):
        if n == 0 :
            return 1 - (self.arrival_rate/self.service_rate)
        else:
            return ((self.arrival_rate/self.service_rate) ** n ) * (1 - (self.arrival_rate/self.service_rate) ) 
    def getCustomer_in_System(self):
        return self.arrival_rate * 1.0 / (self.service_rate - self.arrival_rate)
    def getCustomer_in_Queue(self):
        return (self.arrival_rate ** 2) / self.service_rate * (self.service_rate - self.arrival_rate)
    def getWaitingTime_in_System(self):
        return 1. / (self.service_rate - self.arrival_rate)
    def getWaitingTime_in_Queue(self):
        return self.arrival_rate * 1.0 / self.service_rate * (self.service_rate - self.arrival_rate)

In [9]:
np.random.seed(0)

A = np.array( [
                [0.2, 0.6, 0.2], 
                [0.3, 0.0, 0.7], 
                [0.5, 0.0, 0.5]
               ] )
m  = MarkovChain()
pi_normalized = m.Eigen_Vectors()
# print(m.find_prob([1, 2, 2, 0], A, pi_normalized))



In [10]:

s  = Simulation(2,3)

for i in range(10):

    s.advance_time()
    print("------------------------------------------")
    print("Time : " + str(i)) 
    print("Number of arrival : " + str(s.num_arrivals))
    print("Number of departs : " + str(s.num_departs))
    print("Total wait        : " + str(s.total_wait))
    

print("------------------------------------------")
print("Number of arrival : " + str(s.num_arrivals))
print("Number of departs : " + str(s.num_departs))
print("Total wait        : " + str(s.total_wait))
print("Performance       : " + str(s.total_wait/s.num_departs))
print("------------------------------------------")

------------------------------------------
Time : 0
Number of arrival : 1
Number of departs : 0
Total wait        : 0.0
------------------------------------------
Time : 1
Number of arrival : 1
Number of departs : 1
Total wait        : 0.4186435876552793
------------------------------------------
Time : 2
Number of arrival : 2
Number of departs : 1
Total wait        : 0.4186435876552793
------------------------------------------
Time : 3
Number of arrival : 2
Number of departs : 2
Total wait        : 0.6810439717610363
------------------------------------------
Time : 4
Number of arrival : 3
Number of departs : 2
Total wait        : 0.6810439717610363
------------------------------------------
Time : 5
Number of arrival : 4
Number of departs : 2
Total wait        : 0.9688035713453561
------------------------------------------
Time : 6
Number of arrival : 4
Number of departs : 3
Total wait        : 1.0853905688057894
------------------------------------------
Time : 7
Number of arrival 

In [11]:
A = np.array( [
                [0.2, 0.6, 0.2], 
                [0.3, 0.0, 0.7], 
                [0.5, 0.0, 0.5]
               ] )
B = np.array( [ 
                [0.3, 0.7],
               ])
print(np.matmul(A,B))

[20 24 20]


In [2]:
import timeit
  
  
def print_square(x):
    return (x**2)
  
t = timeit.repeat(lambda: print_square(3), number=10, repeat=5)
  
print(t)

[8.79999998915082e-06, 6.500000012010787e-06, 6.2000000013995304e-06, 6.2000000013995304e-06, 6.299999995462713e-06]
